In [81]:
import requests
import json
import urllib.request
import pandas as pd
from time import sleep

### Country code setting

In [2]:
def country_code_setting(by_code = True):
    base_url = 'https://comtrade.un.org/data/cache/reporterAreas.json'
    r = requests.get(base_url)
    js = json.loads(r.text)
    
    countries = {}
    
    if by_code:
        for lst in js['results']:
            countries[lst['id']] = lst['text']
    else:
        for lst in js['results']:
            countries[lst['text']] = lst['id']
        
    return countries

In [3]:
country_code_byCode = country_code_setting(True)
country_code_byLetter = country_code_setting(False)

#### example

In [4]:
#Country => code
country = input('Which country do you want to get to know?')

print('Country code for {} is {}'.format(country, country_code_byLetter[country]))

Which country do you want to get to know?USA
Country code for USA is 842


In [5]:
#Code => Country
code = input('Which code do you want to get to know?')

print('Code {} is for {}'.format(code, country_code_byCode[code]))

Which code do you want to get to know?842
Code 842 is for USA


## Classification

In [6]:
classification_json = {
    #good default
    'HS': 'https://comtrade.un.org/data/cache/classificationHS.json',
    
    'H0': 'https://comtrade.un.org/data/cache/classificationH0.json',
    'H1':'https://comtrade.un.org/data/cache/classificationH1.json',
    'H2': 'https://comtrade.un.org/data/cache/classificationH2.json',
    'H3': 'https://comtrade.un.org/data/cache/classificationH3.json',
    'H4': 'https://comtrade.un.org/data/cache/classificationH4.json',
    'ST': 'https://comtrade.un.org/data/cache/classificationST.json',
    'S1':'https://comtrade.un.org/data/cache/classificationS1.json',
    'S2':'https://comtrade.un.org/data/cache/classificationS2.json',
    'S3':'https://comtrade.un.org/data/cache/classificationS3.json',
    'S4':'https://comtrade.un.org/data/cache/classificationS4.json',
    'BEC': 'https://comtrade.un.org/data/cache/classificationBEC.json',
    
    #service default
    'EB02': 'https://comtrade.un.org/data/cache/classificationEB02.json'
    
}


In [7]:
def get_class_info(classification_url = classification_json['HS']):
    r = requests.get(classification_url)
    js = json.loads(r.text)
    
    class_info = {}
    
    for info in js['results']:
        class_info[info['id']] = {'text':info['text'], 'parent':info['parent']}
    
    
    return class_info
    

In [8]:
HS = get_class_info()

In [80]:
HS

{'ALL': {'text': 'ALL - All HS commodities', 'parent': '#'},
 'TOTAL': {'text': 'TOTAL - Total of all HS commodities', 'parent': '#'},
 'AG2': {'text': 'AG2 - All 2-digit HS commodities', 'parent': '#'},
 'AG4': {'text': 'AG4 - All 4-digit HS commodities', 'parent': '#'},
 'AG6': {'text': 'AG6 - All 6-digit HS commodities', 'parent': '#'},
 '01': {'text': '01 - Animals; live', 'parent': 'TOTAL'},
 '0101': {'text': '0101 - Horses, asses, mules and hinnies; live',
  'parent': '01'},
 '010110': {'text': '010110 - Horses, asses, mules and hinnies; live, pure-bred breeding animals',
  'parent': '0101'},
 '010111': {'text': '010111 - Horses; live, pure-bred breeding animals',
  'parent': '0101'},
 '010119': {'text': '010119 - Horses; live, other than pure-bred breeding animals',
  'parent': '0101'},
 '010120': {'text': '010120 - Asses, mules and hinnies; live',
  'parent': '0101'},
 '010121': {'text': '010121 - Horses; live, pure-bred breeding animals',
  'parent': '0101'},
 '010129': {'text

In [9]:
HS

{'ALL': {'text': 'ALL - All HS commodities', 'parent': '#'},
 'TOTAL': {'text': 'TOTAL - Total of all HS commodities', 'parent': '#'},
 'AG2': {'text': 'AG2 - All 2-digit HS commodities', 'parent': '#'},
 'AG4': {'text': 'AG4 - All 4-digit HS commodities', 'parent': '#'},
 'AG6': {'text': 'AG6 - All 6-digit HS commodities', 'parent': '#'},
 '01': {'text': '01 - Animals; live', 'parent': 'TOTAL'},
 '0101': {'text': '0101 - Horses, asses, mules and hinnies; live',
  'parent': '01'},
 '010110': {'text': '010110 - Horses, asses, mules and hinnies; live, pure-bred breeding animals',
  'parent': '0101'},
 '010111': {'text': '010111 - Horses; live, pure-bred breeding animals',
  'parent': '0101'},
 '010119': {'text': '010119 - Horses; live, other than pure-bred breeding animals',
  'parent': '0101'},
 '010120': {'text': '010120 - Asses, mules and hinnies; live',
  'parent': '0101'},
 '010121': {'text': '010121 - Horses; live, pure-bred breeding animals',
  'parent': '0101'},
 '010129': {'text

## AVAILABILITY CHECK

In [10]:
def UN_availability(data_type, frequency, reporting_area, time_period, classification):
    base_url = 'http://comtrade.un.org/api//refs/da/view?'
    
    ##조건
    parameters = {'type': data_type,
                 'freq' : frequency,
                 'r' : reporting_area,
                 'ps' : time_period,
                 'px': classification}
    
    url = base_url + dict_to_string(parameters)
    print('url :', url)
    
    r = requests.get(url)
    html = r.text
    js = json.loads(html)
    
    return js 

In [11]:
def dict_item_to_string(key, value):
    """
    inputs: key-value pairs from a dictionary
    output: string 'key=value' or 'key=value1,value2' (if value is a list)
    examples: 'fmt', 'csv' => 'fmt=csv' or 'r', [124, 484] => 'r=124,484'
    """
    value_string = str(value) if not isinstance(value, list) else ','.join(map(str, value))
    return '='.join([key, value_string])


def dict_to_string(parameters):
    """
    input: dictionary of parameters
    output: string 'key1=value1&key2=value2&...'
    """
    param =[]
    for key, value in parameters.items():
        if value != 'any':
            param.append(dict_item_to_string(key, value))
    return '&'.join(param)


In [77]:
ava = UN_availability('C','any', 'any','any','HS')

url : http://comtrade.un.org/api//refs/da/view?type=C&freq=any&r=any&ps=any&px=HS


### TO DATAFRAME

In [78]:
avail_df = pd.DataFrame(ava)

In [79]:
avail_df

""


In [19]:
avail_df[(avail_df['rDesc'] =='USA')&(avail_df['freq'] == 'ANNUAL')]

,TotalRecords,freq,isOriginal,isPartnerDetail,ps,publicationDate,px,r,rDesc,type
711,504309.0,ANNUAL,0,1,1991,2005-12-23T00:00:00,HS,842,USA,COMMODITIES
712,519576.0,ANNUAL,0,1,1992,2005-12-30T00:00:00,HS,842,USA,COMMODITIES
713,537071.0,ANNUAL,0,1,1993,2005-12-30T00:00:00,HS,842,USA,COMMODITIES
714,550649.0,ANNUAL,0,1,1994,2006-01-09T00:00:00,HS,842,USA,COMMODITIES
715,569048.0,ANNUAL,0,1,1995,2002-09-01T00:00:00,HS,842,USA,COMMODITIES
1590,586057.0,ANNUAL,0,1,1996,2006-01-09T00:00:00,HS,842,USA,COMMODITIES
1591,609454.0,ANNUAL,0,1,1997,2006-01-09T00:00:00,HS,842,USA,COMMODITIES
1592,606635.0,ANNUAL,0,1,1998,2005-12-30T00:00:00,HS,842,USA,COMMODITIES
1593,605685.0,ANNUAL,0,1,1999,2005-12-28T00:00:00,HS,842,USA,COMMODITIES
1594,632861.0,ANNUAL,0,1,2000,2018-01-08T00:00:00,HS,842,USA,COMMODITIES


## UN COMTRADE DATA REQUEST

* API의 parameters  
1) reporting country: 수출/수입을 UN에 보고하는 국가  
2) frequency: 연간, 월간  
3) classification : HS, SITC  
4) partner country: reporting country의 상대국가  
5) trade flow: 수출/수입/재수출/재수입  
6) commodity code: 2 digit/ 4 digit / 6 digit(5 digit)  

In [20]:
def UN_comtrade(human_readable=False, verbose=True,
    period='recent', frequency='A', reporter=842, partner='all', product='total', tradeflow=2):

    global country_code_byLetter
    base_url = 'https://comtrade.un.org/api/get?'
    
    ##조건
    fmt = 'csv' if human_readable else 'json'
    head = 'H' if human_readable else 'M'
    
    if type(reporter) == str:
        reporter = country_code_byLetter[reporter]
    
    parameters = {
            'ps': period,
            'freq': frequency,
            'r': reporter,
            'p': partner,
            'cc': product,
            'rg': tradeflow,
            'px': 'HS',      # Harmonized System (as reported) as classification scheme
            'type': 'C',     # Commodities ('S' for Services)
            'fmt': fmt,      # format of the output
            'max': 50000,    # maximum number of rows -> what happens if number of rows is bigger?
                             # https://comtrade.un.org/data/dev/portal#subscription says it is 100 000
            'head': head     # human readable headings ('H') or machine readable headings ('M')
        }
    
    url = base_url + dict_to_string(parameters)
    print('url :', url)
    
    r = requests.get(url)
    html = r.text
    js = json.loads(html)
    
    if js['validation']['message']:
        print(js['validation']['message'])
    
    print('\n# of data is', js['validation']['count']['value'])
    
    return js 

In [21]:
def dict_item_to_string(key, value):
    """
    inputs: key-value pairs from a dictionary
    output: string 'key=value' or 'key=value1,value2' (if value is a list)
    examples: 'fmt', 'csv' => 'fmt=csv' or 'r', [124, 484] => 'r=124,484'
    """
    value_string = str(value) if not isinstance(value, list) else ','.join(map(str, value))
    return '='.join([key, value_string])


def dict_to_string(parameters):
    """
    input: dictionary of parameters
    output: string 'key1=value1&key2=value2&...'
    """
    return '&'.join(dict_item_to_string(key, value) for key, value in parameters.items())

In [22]:
data = UN_comtrade(period = 1991, reporter = 'USA')

url : https://comtrade.un.org/api/get?ps=1991&freq=A&r=842&p=all&cc=total&rg=2&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 194


In [65]:
data17 = UN_comtrade(period = 2017, reporter = 'USA')

url : https://comtrade.un.org/api/get?ps=2017&freq=A&r=842&p=all&cc=total&rg=2&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 225


### TO DATAFRAME

In [23]:
trade_df = pd.DataFrame(data['dataset'])

In [24]:
trade_df.columns

Index(['AltQuantity', 'CIFValue', 'FOBValue', 'GrossWeight', 'IsLeaf',
       'NetWeight', 'TradeQuantity', 'TradeValue', 'aggrLevel', 'cmdCode',
       'cmdDescE', 'cstCode', 'cstDesc', 'estCode', 'motCode', 'motDesc',
       'period', 'periodDesc', 'pfCode', 'pt3ISO', 'pt3ISO2', 'ptCode',
       'ptCode2', 'ptTitle', 'ptTitle2', 'qtAltCode', 'qtAltDesc', 'qtCode',
       'qtDesc', 'rgCode', 'rgDesc', 'rt3ISO', 'rtCode', 'rtTitle', 'yr'],
      dtype='object')

In [71]:
trade1701 = pd.DataFrame(data1701['dataset'])

In [75]:
data1701

{'validation': {'status': {'name': 'Result too large',
   'value': 5003,
   'category': 0,
   'description': 'you do not have permissions to access such a large resultset.',
   'helpUrl': 'For more reference visit http://comtrade.un.org/data/dev/portal/'},
  'message': 'Maximum resultset is: 100000',
  'count': {'value': 136531,
   'started': '0001-01-01T00:00:00',
   'finished': '0001-01-01T00:00:00',
   'durationSeconds': 0.0},
  'datasetTimer': {'started': '2019-01-29T23:54:27.1451772-05:00',
   'finished': '2019-01-29T23:54:47.7071772-05:00',
   'durationSeconds': 20.561999999999998}},
 'dataset': []}

In [30]:
trade17 = pd.DataFrame(data['dataset'])

In [56]:
not_null = []

for col in trade17.columns:
    if trade17[trade17[col].isnull() == False].shape[0]:
        not_null.append(col)

print(not_null)

['IsLeaf', 'TradeValue', 'aggrLevel', 'cmdCode', 'cmdDescE', 'cstCode', 'cstDesc', 'estCode', 'motCode', 'motDesc', 'period', 'periodDesc', 'pfCode', 'pt3ISO', 'pt3ISO2', 'ptCode', 'ptTitle', 'ptTitle2', 'qtAltDesc', 'qtCode', 'qtDesc', 'rgCode', 'rgDesc', 'rt3ISO', 'rtCode', 'rtTitle', 'yr']


In [68]:
trade1701

,AltQuantity,CIFValue,FOBValue,GrossWeight,IsLeaf,NetWeight,TradeQuantity,TradeValue,aggrLevel,cmdCode,...,qtAltCode,qtAltDesc,qtCode,qtDesc,rgCode,rgDesc,rt3ISO,rtCode,rtTitle,yr
0,None,None,None,None,0,None,None,128142016091,0,TOTAL,...,None,,0,None,2,Exports,None,842,United States of America,2015
1,None,None,None,None,0,None,None,29053276,0,TOTAL,...,None,,0,None,2,Exports,None,842,United States of America,2015
2,None,None,None,None,0,None,None,3096707,0,TOTAL,...,None,,0,None,2,Exports,None,842,United States of America,2015
3,None,None,None,None,0,None,None,140743982,0,TOTAL,...,None,,0,None,2,Exports,None,842,United States of America,2015
4,None,None,None,None,0,None,None,102130,0,TOTAL,...,None,,0,None,2,Exports,None,842,United States of America,2015
5,None,None,None,None,0,None,None,123063590,0,TOTAL,...,None,,0,None,2,Exports,None,842,United States of America,2015
6,None,None,None,None,0,None,None,45648614,0,TOTAL,...,None,,0,None,2,Exports,None,842,United States of America,2015
7,None,None,None,None,0,None,None,22213704,0,TOTAL,...,None,,0,None,2,Exports,None,842,United States of America,2015
8,None,None,None,None,0,None,None,934967183,0,TOTAL,...,None,,0,None,2,Exports,None,842,United States of America,2015
9,None,None,None,None,0,None,None,2214411616,0,TOTAL,...,None,,0,None,2,Exports,None,842,United States of America,2015


In [59]:
trade_df['cmdCode']

0      TOTAL
1      TOTAL
2      TOTAL
3      TOTAL
4      TOTAL
5      TOTAL
6      TOTAL
7      TOTAL
8      TOTAL
9      TOTAL
10     TOTAL
11     TOTAL
12     TOTAL
13     TOTAL
14     TOTAL
15     TOTAL
16     TOTAL
17     TOTAL
18     TOTAL
19     TOTAL
20     TOTAL
21     TOTAL
22     TOTAL
23     TOTAL
24     TOTAL
25     TOTAL
26     TOTAL
27     TOTAL
28     TOTAL
29     TOTAL
       ...  
164    TOTAL
165    TOTAL
166    TOTAL
167    TOTAL
168    TOTAL
169    TOTAL
170    TOTAL
171    TOTAL
172    TOTAL
173    TOTAL
174    TOTAL
175    TOTAL
176    TOTAL
177    TOTAL
178    TOTAL
179    TOTAL
180    TOTAL
181    TOTAL
182    TOTAL
183    TOTAL
184    TOTAL
185    TOTAL
186    TOTAL
187    TOTAL
188    TOTAL
189    TOTAL
190    TOTAL
191    TOTAL
192    TOTAL
193    TOTAL
Name: cmdCode, Length: 194, dtype: object